In [ ]:
import requests 

# Getting the DATA 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

##  Q1. Run Elastic Search 8.17.6, and get the cluster information. 

If you run it on `localhost`, this is how you do it:

```curl localhost:9200```

 What's the version.build_hash value?



In [90]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

## Q2. Indexing the data

 Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.
### Don't forget to install the ElasticSearch client for Python:

```pip install elasticsearch```

 Which function do you use for adding your data to elastic?

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-faq"

# create index
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    
    # Index each document in Elasticsearch
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

- Now let's search in our index.
- We will execute a query "How do execute a command on a Kubernetes pod?".
- Use only ```question``` and ```text``` fields and give question a boost of ```4```, and use ```"type": "best_fields"```
- What's the score for the top ranking result?
- Look at the ```_score``` field.

In [ ]:
query = 'How do execute a command on a Kubernetes pod?'
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

In [ ]:
def elastic_search(query, search_query):

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    
    
    for hit in response['hits']['hits']:
        result_docs.append({
            "question": hit['_source'].get("question", ""),
            "text": hit['_source'].get("text", ""),
            "score": hit['_score']
        })
    
    return result_docs

In [ ]:
print(elastic_search(query, search_query))

[{'question': 'How do I debug a docker container?', 'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'score': 73.38676}, {'question': 'How do I copy files from my local machine to docker container?', 'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'score': 66.688705}, {'question': 'How do I copy files from a different folder into docker container’s working directory?', 'text': 'Y

# Q4. Now ask a different question: "How do copy a file to a Docker container?".
 This time we are only interested in questions from ```machine-learning-zoomcamp```.
 Return 3 results. 
 What's the 3rd question returned by the search engine?

In [ ]:

query = 'How do copy a file to a Docker container?'
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [ ]:
# print(elastic_search(query, search_query))
search_results = elastic_search(query, search_query)

In [ ]:
def build_context(search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = ""
    context_entries = []

    for result in search_results:
        # print(f"Question: {result['question']}")
        # print(f"Text: {result['text']}")
        entry = context_template.format(question=result['question'], text=result['text'])
        context_entries.append(entry)

    context = "\n\n".join(context_entries)

    return context

In [ ]:
from openai import OpenAI
import os

In [ ]:
os.environ["OPENAI_API_KEY"]="secret-key"


In [ ]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
# Q5 : What's the length of the resulting prompt? 
print(query)
context = build_context(search_results)
print(context)
prompt = build_prompt(query, search_results)
print(f"Length of the prompt: {len(prompt)} characters")

How do copy a file to a Docker container?
Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
    A: You can copy files from your local machine into a Docker

## Q6. When we use the OpenAI Platform,
 we're charged by the number of tokens we send in our prompt 
 and receive in the response.

- Let's calculate the number of tokens in our query:

```encoding = tiktoken.encoding_for_model("gpt-4o")```

 Use the encode function. How many tokens does our prompt have?


In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tiktoken]


In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
num_tokens = len(encoding.encode(prompt))
print(f"Number of tokens in the prompt: {num_tokens}")


Number of tokens in the prompt: 327
